In [53]:
import torch

from core.utils.data_utils import prepare_datasets, DataStream, vectorize_input
from core.model_handler import ModelHandler, batch_diff
from core.utils import padding_utils
from main import get_config
from core.model import Model

In [37]:
fpath_config = "/home/gaurang/IDGL/src/config/mrd/idgl.yml"

In [38]:
config = get_config(fpath_config)
# config["device"] = torch.device('cpu')

In [39]:
datasets = prepare_datasets(config)

[ Max seq length: 2366 ]
[ Min seq length: 6 ]
[ Mean seq length: 389 ]
# of training examples: 3003
# of dev examples: 1001
# of testing examples: 1002


In [40]:
train_set = datasets['train']
dev_set = datasets['dev']
test_set = datasets['test']

In [41]:
model_handler = ModelHandler(config)

[ Using CUDA ]
[ Max seq length: 2366 ]
[ Min seq length: 6 ]
[ Mean seq length: 389 ]
# of training examples: 3003
# of dev examples: 1001
# of testing examples: 1002
[ Running TextGraphRegression model ]
Loading pre-built vocab model stored in ../data/mrd/mrd_seed1234.vocab
[ Fix word embeddings ]
[ Using 1-layer bidirectional lstm encoder ]
[ Using TextGNN ]
[ GAT_Attention GraphLearner]
[ Graph Learner metric type: gat_attention ]
[ GAT_Attention GraphLearner]
[ Graph Learner metric type: gat_attention ]
[ Graph Learner ]
[ Graph Regularization]
word_embed.weight: torch.Size([7871, 300])
ctx_rnn_encoder.model.weight_ih_l0: torch.Size([128, 300])
ctx_rnn_encoder.model.weight_hh_l0: torch.Size([128, 32])
ctx_rnn_encoder.model.bias_ih_l0: torch.Size([128])
ctx_rnn_encoder.model.bias_hh_l0: torch.Size([128])
ctx_rnn_encoder.model.weight_ih_l0_reverse: torch.Size([128, 300])
ctx_rnn_encoder.model.weight_hh_l0_reverse: torch.Size([128, 32])
ctx_rnn_encoder.model.bias_ih_l0_reverse: torch

In [42]:
input_batch = model_handler.train_loader.nextBatch()

In [43]:
x_batch = vectorize_input(input_batch, config, training=True)

In [44]:
context, context_lens, targets = x_batch['context'], x_batch['context_lens'], x_batch['targets']

In [45]:
context.shape

torch.Size([16, 364])

In [46]:
network = model_handler.model.network
raw_context_vec, context_vec, context_mask, init_adj = network.prepare_init_graph(context, context_lens)

In [47]:
init_adj.shape

torch.Size([16, 364, 364])

In [48]:
batch_stop_indicators = torch.ones(x_batch['batch_size'], dtype=torch.uint8)

In [49]:
node_mask = context_mask
raw_node_vec = raw_context_vec
cur_raw_adj, cur_adj = network.learn_graph(network.graph_learner, raw_node_vec, network.graph_skip_conn, node_mask=node_mask, graph_include_self=network.graph_include_self, init_adj=init_adj)

In [59]:
eps_adj = float(config.get('eps_adj', 0))
tmp_stop_criteria = batch_diff(cur_adj, cur_adj, init_adj) > eps_adj

In [62]:
batch_stop_indicators = batch_stop_indicators * tmp_stop_criteria
torch.sum(batch_stop_indicators).item() > 0

False